# Implement our own system assembling

In this tutorial we

* write integrators for $\int_T f v dx$ and $\int_T \nabla u \nabla v dx$: 
* put together element matrices to the global system matrix: 


The implementation is available on github: 
[https://github.com/TUWien-ASC/NGS-myassembling](https://github.com/TUWien-ASC/NGS-myassembling)



In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
import myassembling
dir (myassembling)

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
fes = H1(mesh, order=2, dirichlet=".*")
u, v = fes.TnT()

### use our own integrators for element matrix calculation:

In [ ]:
a = BilinearForm(fes)
a += myassembling.MyLaplace(1)
a.Assemble()

f = LinearForm(fes)
f += myassembling.MySource(x)
f.Assemble();

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw (gfu);

### we can inspect the element matrix:

In [ ]:
mylap = myassembling.MyLaplace(1)
ei = ElementId(VOL,17)
mylap.CalcElementMatrix(fes.GetFE(ei), mesh.GetTrafo(ei))

### use our own matrix assembling function:

In [ ]:
for l in range(4): mesh.Refine()
fes.Update()
gfu.Update()
f.Assemble();

In [ ]:
mymatrix = myassembling.MyAssembleMatrix(fes, myassembling.MyLaplace(CF(1)))
# mymatrix = myassembling.MyAssembleMatrix(fes, (grad(u)*grad(v)*dx)[0].MakeBFI())

if fes.ndof < 100000:
    gfu.vec.data = mymatrix.Inverse(fes.FreeDofs()) * f.vec
    Draw (gfu);

### Exercise:

   * Implement `MyAssembleVector` for building the right hand side vector for  $\int_\Omega f v \, dx$
   * Implement `MyNeumannIntegrator` for evaluating $\int_{\partial \Omega} g v \, ds$

   * Have a look into thread-parallel assembling at [NGSolve-itutorials unit 9.2](https://docu.ngsolve.org/latest/i-tutorials/unit-9.2-C%2B%2BAssemble/cppassembling.html)